In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,119 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [910 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ge

In [4]:
#Import dependencies
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Import Keras Dependencies
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [6]:
 # Import findspark and initialize.
import findspark
findspark.init()

In [7]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("weatherclassification").getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [8]:
# Load in data
from pyspark import SparkFiles
url = 'https://weatherclassification.s3.amazonaws.com/weather_classification_data.csv'
spark.sparkContext.addFile(url)
print(SparkFiles.getRootDirectory())

spark_df = spark.read.csv(SparkFiles.get("weather_classification_data.csv"), sep=",", header=True, inferSchema=True)
spark_df.show()

/tmp/spark-6b821ec8-c8e0-4a38-8745-e1de21c76e36/userFiles-ca80bf0b-363f-49ac-983a-39f2302978e5
+-----------+--------+----------+-----------------+-------------+--------------------+--------+------+---------------+--------+------------+
|Temperature|Humidity|Wind Speed|Precipitation (%)|  Cloud Cover|Atmospheric Pressure|UV Index|Season|Visibility (km)|Location|Weather Type|
+-----------+--------+----------+-----------------+-------------+--------------------+--------+------+---------------+--------+------------+
|         14|      73|       9.5|               82|partly cloudy|             1010.82|       2|Winter|            3.5|  inland|       Rainy|
|         39|      96|       8.5|               71|partly cloudy|             1011.43|       7|Spring|           10.0|  inland|      Cloudy|
|         30|      64|       7.0|               16|        clear|             1018.72|       5|Spring|            5.5|mountain|       Sunny|
|         38|      83|       1.5|               82|        

In [9]:
# Convert PySpark DataFrame to pandas DataFrame
weather_df = spark_df.toPandas()
weather_df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14,73,9.5,82,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39,96,8.5,71,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30,64,7.0,16,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38,83,1.5,82,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27,74,17.0,66,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [10]:
# Investigate the value counts
weather_df['Weather Type'].value_counts()

Weather Type
Rainy     3300
Cloudy    3300
Sunny     3300
Snowy     3300
Name: count, dtype: int64

In [11]:
#Create weather data remove seasons
weather_data_no_type = weather_df.drop('Weather Type', axis=1)

In [12]:
# Convert categorical data to numeric with `StringIndexer`
weather_data_no_type = pd.get_dummies(weather_data_no_seasons).astype(int)
weather_data_no_type.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km),Cloud Cover_clear,Cloud Cover_cloudy,Cloud Cover_overcast,Cloud Cover_partly cloudy,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Location_coastal,Location_inland,Location_mountain
0,14,73,9,82,1010,2,3,0,0,0,1,0,0,0,1,0,1,0
1,39,96,8,71,1011,7,10,0,0,0,1,0,1,0,0,0,1,0
2,30,64,7,16,1018,5,5,1,0,0,0,0,1,0,0,0,0,1
3,38,83,1,82,1026,7,1,1,0,0,0,0,1,0,0,1,0,0
4,27,74,17,66,990,1,2,0,0,1,0,0,0,0,1,0,0,1


In [13]:
# Split our preprocessed data into our features and target arrays
X = weather_data_no_type
y = weather_df['Weather Type']

# Use LabelEncoder to convert categorical season labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = 4
y_one_hot = to_categorical(y_encoded, num_classes)

encoded_labels = label_encoder.classes_
encoded_labels = label_encoder.transform(label_encoder.classes_)
label_mapping = dict(zip(encoded_labels, label_encoder.classes_))
print(f"Labels: {label_mapping}")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, random_state=1,stratify = y_one_hot)

Labels: {0: 'Cloudy', 1: 'Rainy', 2: 'Snowy', 3: 'Sunny'}


In [14]:
# Normalize Data
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Investigate Data Shape
print('Data Shapes')
print(f'Training Data: {X_train.shape}')
print(f'Training Labels: {y_train.shape}')
print(f'Training Data: {X_test.shape}')
print(f'Training Labels: {y_test.shape}')

Data Shapes
Training Data: (9900, 18)
Training Labels: (9900, 4)
Training Data: (3300, 18)
Training Labels: (3300, 4)


In [16]:
# Create function for model creation
def gen_model(units, activation_functions, input_dim, optimizer):
  # Define the deep learning model
  nn_model = tf.keras.models.Sequential()
  nn_model.add(tf.keras.Input(shape=(input_dim,)))
  for i in range(len(units)):
    nn_model.add(tf.keras.layers.Dense(units=units[i], activation=activation_functions[i]))
  nn_model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

  # Compile the Sequential model together and customize metrics
  nn_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
  return nn_model

In [17]:
# Model 1
units = [100, 50, 50, 25]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'adam')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 1 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6759 - loss: 0.8404
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8880 - loss: 0.3314
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8940 - loss: 0.2846
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9003 - loss: 0.2663
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9065 - loss: 0.2499
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9072 - loss: 0.2389
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9014 - loss: 0.2380
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9076 - loss: 0.2297
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9074 - loss: 0.2352
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9082 - loss: 0.2182
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9148 - loss: 0.2133
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

In [18]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 1')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 1")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))

104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Confusion Matrix Model 1
[[719  45  22  39]
 [ 25 744  31  25]
 [ 12  11 771  31]
 [ 27  27  24 747]]
Classification Report Model 1
              precision    recall  f1-score   support

      Autumn       0.92      0.87      0.89       825
      Spring       0.90      0.90      0.90       825
      Summer       0.91      0.93      0.92       825
      Winter       0.89      0.91      0.90       825

    accuracy                           0.90      3300
   macro avg       0.90      0.90      0.90      3300
weighted avg       0.90      0.90      0.90      3300



In [19]:
# Model 2 - Increase neurons
units = [200, 100, 100, 50]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'adam')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 2 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7379 - loss: 0.7360
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8852 - loss: 0.3177
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8963 - loss: 0.2758
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8982 - loss: 0.2487
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9058 - loss: 0.2397
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9081 - loss: 0.2292
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9092 - loss: 0.2221
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9157 - loss: 0.2070
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9126 - loss: 0.2083
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9177 - loss: 0.2069
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9173 - loss: 0.1913
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

In [20]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 2')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 2")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix Model 2
[[724  55  16  30]
 [ 26 759  17  23]
 [ 15  21 764  25]
 [ 29  28  13 755]]
Classification Report Model 2
              precision    recall  f1-score   support

      Cloudy       0.91      0.88      0.89       825
       Rainy       0.88      0.92      0.90       825
       Snowy       0.94      0.93      0.93       825
       Sunny       0.91      0.92      0.91       825

    accuracy                           0.91      3300
   macro avg       0.91      0.91      0.91      3300
weighted avg       0.91      0.91      0.91      3300



In [21]:
# Model 3 - Increase neurons
units = [400, 200, 200, 100]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'adam')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 3 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7639 - loss: 0.6674
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8854 - loss: 0.3075
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9015 - loss: 0.2581
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9037 - loss: 0.2429
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8977 - loss: 0.2480
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9037 - loss: 0.2282
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9127 - loss: 0.2095
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9131 - loss: 0.2102
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9147 - loss: 0.2059
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9088 - loss: 0.2151
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9242 - loss: 0.1821
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

In [22]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 3')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 3")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))

104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Confusion Matrix Model 3
[[734  48  22  21]
 [ 78 712  20  15]
 [ 14  27 762  22]
 [ 37  36  18 734]]
Classification Report Model 3
              precision    recall  f1-score   support

      Cloudy       0.85      0.89      0.87       825
       Rainy       0.87      0.86      0.86       825
       Snowy       0.93      0.92      0.93       825
       Sunny       0.93      0.89      0.91       825

    accuracy                           0.89      3300
   macro avg       0.89      0.89      0.89      3300
weighted avg       0.89      0.89      0.89      3300



In [23]:
# Model 4
units = [100, 50, 50, 25]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'SGD')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 4 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4425 - loss: 1.3374
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6724 - loss: 1.0072
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7673 - loss: 0.6647
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8262 - loss: 0.5742
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8492 - loss: 0.5052
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8559 - loss: 0.4765
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8644 - loss: 0.4395
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8759 - loss: 0.4121
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8761 - loss: 0.3989
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8805 - loss: 0.3867
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.3920
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

In [24]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 4')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 4")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix Model 4
[[736  54  13  22]
 [ 45 748  10  22]
 [ 25  15 761  24]
 [ 47  30  11 737]]
Classification Report Model 4
              precision    recall  f1-score   support

      Cloudy       0.86      0.89      0.88       825
       Rainy       0.88      0.91      0.89       825
       Snowy       0.96      0.92      0.94       825
       Sunny       0.92      0.89      0.90       825

    accuracy                           0.90      3300
   macro avg       0.90      0.90      0.90      3300
weighted avg       0.90      0.90      0.90      3300



In [25]:
# Model 5
units = [200, 100, 100, 50]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'Adagrad')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 5 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2860 - loss: 1.3772
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6015 - loss: 1.3233
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6646 - loss: 1.2543
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6943 - loss: 1.1500
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7113 - loss: 1.0246
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7264 - loss: 0.8913
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7383 - loss: 0.8008
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7467 - loss: 0.7313
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7579 - loss: 0.6854
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7704 - loss: 0.6580
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7997 - loss: 0.6142
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [26]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 5')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 5")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix Model 5
[[723  67  12  23]
 [ 58 690  64  13]
 [ 37  12 754  22]
 [ 61  24   6 734]]
Classification Report Model 5
              precision    recall  f1-score   support

      Cloudy       0.82      0.88      0.85       825
       Rainy       0.87      0.84      0.85       825
       Snowy       0.90      0.91      0.91       825
       Sunny       0.93      0.89      0.91       825

    accuracy                           0.88      3300
   macro avg       0.88      0.88      0.88      3300
weighted avg       0.88      0.88      0.88      3300



In [ ]:
# Model 6
units = [400, 200, 200, 100]
activations_funcs = ['relu', 'relu', 'relu', 'relu']
nn_model = gen_model(units, activations_funcs, 18, 'RMSProp')


# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Model 6 Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7369 - loss: 0.6795
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8704 - loss: 0.3403
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8817 - loss: 0.2969
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8957 - loss: 0.2683
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8991 - loss: 0.2422
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9082 - loss: 0.2352
Epoch 7/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9085 - loss: 0.2356
Epoch 8/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9141 - loss: 0.2145
Epoch 9/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9142 - loss: 0.2170
Epoch 10/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9162 - loss: 0.2049
Epoch 11/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9142 - loss: 0.2099
Epoch 12/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

In [ ]:
# Make predictions using testing data
predictions = nn_model.predict(X_test)

# Convert one-hot encoded values to categorical labes
y_test_labels = np.argmax(y_test, axis = 1)

predictions_labels = np.argmax(predictions, axis=1)

# Confusion Matrix
print('Confusion Matrix Model 6')
print(confusion_matrix(y_test_labels, predictions_labels))

# Print the classification report for the model
print("Classification Report Model 6")
target_names = ["Cloudy", "Rainy", "Snowy", "Sunny"]
print(classification_report(y_test_labels, predictions_labels, target_names=target_names))